# Borda Count NF-BoT-IoT-v2

In [2]:
from itertools import chain
from pathlib import Path
import json
import pandas as pd

def convert(path):
    f = open(path)
    name_file = Path(f.name).stem

    name_file = name_file.replace('ExtraTreesClassifier','ET').replace('GaussianNB','NB').replace('DecisionTreeClassifier','DT')\
    .replace('RandomForestClassifier','RF').replace('LGBMClassifier','LB').replace('XGBClassifier','XB').replace('X_','').replace('A_mRMR_','')

    data_algo = name_file.split('_')

    final_name = f'{data_algo[0]}_{data_algo[-2:-1][0]}_{data_algo[-1:][0]}'
    final_name = final_name.replace('__X','100')
    final_name = final_name.replace('__','')


    data = json.load(f)
    accuracies = data['mean_test_Accuracy']
    f1_scores = data['mean_test_F1-Score']
    mccs = data['mean_test_MCC']
    kappass = data['mean_test_Kappa']
    params = data['params']
    zipSorted=sorted(zip(accuracies,f1_scores,mccs,kappass, params),key = lambda t: t[0],reverse=True)
    my_dict=params
    my_data=[]
    for accuracy, f1, recall, precision, param in zipSorted:
        score={"Algo":final_name ,"Accuracy":accuracy,"F1-Score":f1,'MCC':recall , 'Kappa':precision }
        my_data_tmp =  dict(chain.from_iterable(d.items() for d in (score, param)))
        my_data.append(my_data_tmp)
    return final_name , pd.DataFrame(data=my_data)

In [3]:
from os import listdir
from os.path import isfile, join
mypath = './result_multi_bot'
my_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
my_result_mean = {}
for f in my_files:
  path_file = join(mypath, f)
  k,v = convert(path_file)
  my_result_mean[k] = v

In [4]:
my_result_mean.keys()

dict_keys(['DT100', 'DT20', 'DT40', 'DT60', 'ET100', 'ET20', 'ET40', 'ET60', 'NB100', 'NB20', 'NB40', 'NB60', 'MLP100', 'MLP20', 'MLP40', 'MLP60', 'RF100', 'RF20', 'RF40', 'RF60', 'XB100', 'XB20', 'XB40', 'XB60'])

In [5]:
key_list = [ 'ET20', 'ET40', 'ET60','ET100',  'RF20', 'RF40', 'RF60', 'RF100', 'XB20', 'XB40', 'XB60', 'XB100']
my_list = []
filtered_dict_ensemble = {key: value for key, value in my_result_mean.items() if key in key_list}
for k,v in filtered_dict_ensemble.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_E{x['n_estimators']}D{x['max_depth']}", axis=1)
    filtered_dict_ensemble[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_ensemble[k])

df_ensemble = pd.concat(my_list)

In [6]:
key_list = ['DT100', 'DT20', 'DT40', 'DT60']
my_list = []

filtered_dict_DT = {key: value for key, value in my_result_mean.items() if key in key_list}

for k,v in filtered_dict_DT.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_D{x['max_depth']}", axis=1)
    filtered_dict_DT[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_DT[k])

df_DT = pd.concat(my_list)

In [7]:
key_list = ['MLP100', 'MLP20', 'MLP40', 'MLP60']

my_list = []

filtered_dict_MLP = {key: value for key, value in my_result_mean.items() if key in key_list}

for k,v in filtered_dict_MLP.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_L{x['hidden_layer_sizes']}", axis=1)
    filtered_dict_MLP[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_MLP[k])

df_MLP = pd.concat(my_list)


In [8]:
key_list = ['NB100', 'NB20', 'NB40', 'NB60']

my_list = []

filtered_dict_NB = {key: value for key, value in my_result_mean.items() if key in key_list}

for k,v in filtered_dict_NB.items():
    v['Algo'] = v.apply(lambda x: f"{x['Algo']}_L{x['var_smoothing']}", axis=1)
    filtered_dict_NB[k] = v[['Algo','MCC','Kappa']]
    my_list.append(filtered_dict_NB[k])

df_NB = pd.concat(my_list)


In [9]:

my_list = [df_ensemble,df_DT,df_MLP,df_NB]
df_bot = pd.concat(my_list)


The first cluster contains all six MLP models, two DT models (DT20_D12 and DT20_D4), and one model each of ET, RF, and XGB (ET20_E2D16, RF20_E4D8, and XGB20_E2D8). 

In [10]:
cluster_1_20_bot = ['MLP20_L56','MLP20_L46','MLP20_L36', 'MLP20_L26', 'MLP20_L16', 'MLP20_L6','DT20_D12','DT20_D4','ET20_E2D16','RF20_E4D8','XB20_E2D8']

cluster_1_20_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_20_bot)]


In [11]:
cluster_1_20_bot_df

,Algo,MCC,Kappa
1,ET20_E2D16,0.974420,0.974416
3,RF20_E4D8,0.971213,0.971103
5,XB20_E2D8,0.973496,0.973490
4,DT20_D12,0.975108,0.975103
5,DT20_D4,0.971328,0.971308
0,MLP20_L56,0.973024,0.973009
1,MLP20_L36,0.972981,0.972966
2,MLP20_L16,0.972903,0.972886
3,MLP20_L46,0.972778,0.972761
4,MLP20_L26,0.972759,0.972742


In [12]:
bot_arduino = pd.read_excel('bot_arduino.xlsx')
bot_arduino

def rename_algo(x):
    my_list = x.split('_')
    return f"{my_list[0]}{my_list[1]}_{my_list[2]}"

bot_arduino['Algo'] = bot_arduino['Algo'].apply(lambda x : rename_algo(x) )

In [1]:
print("hello world!!")

hello world!!


In [13]:
bot_arduino_cluster_1_20 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_20_bot)]

df_bot_all_20 = pd.merge(cluster_1_20_bot_df, bot_arduino_cluster_1_20, on='Algo', how='inner')


df_bot_all_20

,Algo,MCC,Kappa,Latency,SRAM,FM
0,ET20_E2D16,0.974420,0.974416,8.0,4730,353
1,RF20_E4D8,0.971213,0.971103,145.6,12948,432
2,XB20_E2D8,0.973496,0.973490,332.0,31162,432
3,DT20_D12,0.975108,0.975103,50.4,13896,432
4,DT20_D4,0.971328,0.971308,24.0,3694,432
5,MLP20_L56,0.973024,0.973009,9.6,6330,446
6,MLP20_L36,0.972981,0.972966,9.6,6330,446
7,MLP20_L16,0.972903,0.972886,8.8,6330,446
8,MLP20_L46,0.972778,0.972761,8.8,6330,446
9,MLP20_L26,0.972759,0.972742,9.6,6330,446


## Borda Count

In [14]:

import numpy as np
def borda_count(my_data_borda):
  rank_points = my_data_borda[['Kappa','MCC', 'Latency', 'SRAM', 'FM']].values.argsort(axis=0).argsort(axis=0)
 
  borda_counts = rank_points.sum(axis=1)

  my_data_borda['Borda'] = borda_counts
  sorted_scores = my_data_borda.sort_values(by=['Borda', 'Kappa'], ascending=False)

  return sorted_scores

In [15]:
import numpy as np
import pandas as pd

def borda_count(my_data_borda):
    # Define the weights
    weights = {'Kappa': 2.5, 'MCC': 2.5, 'Latency':2, 'SRAM': 1.5, 'FM': 1}

    # Extract the columns to be ranked
    columns = ['Kappa', 'MCC', 'Latency', 'SRAM', 'FM']
    
    # Calculate rank points
    rank_points = my_data_borda[columns].values.argsort(axis=0).argsort(axis=0)

    # Apply weights to the rank points
    weighted_rank_points = np.zeros(rank_points.shape)
    for i, col in enumerate(columns):
        weighted_rank_points[:, i] = rank_points[:, i] * weights[col]
    
    # Sum the weighted rank points
    borda_counts = weighted_rank_points.sum(axis=1)

    # Assign the Borda counts to a new column and sort
    my_data_borda['Borda'] = borda_counts
    sorted_scores = my_data_borda.sort_values(by=['Borda', 'Kappa'], ascending=False)

    return sorted_scores

In [16]:

df_bot_all_20_tmp = df_bot_all_20.copy()
df_bot_all_20_tmp[['Latency', 'SRAM', 'FM']] = -df_bot_all_20[['Latency', 'SRAM', 'FM']]
df_bot_all_20_bc = borda_count(df_bot_all_20_tmp)
df_bot_all_20_bc[['Latency', 'SRAM', 'FM']] = -df_bot_all_20_bc[['Latency', 'SRAM', 'FM']]
df_bot_all_20_bc

,Algo,MCC,Kappa,Latency,SRAM,FM,Borda
0,ET20_E2D16,0.974420,0.974416,8.0,4730,353,88.5
3,DT20_D12,0.975108,0.975103,50.4,13896,432,63.5
7,MLP20_L16,0.972903,0.972886,8.8,6330,446,50.5
8,MLP20_L46,0.972778,0.972761,8.8,6330,446,50.0
5,MLP20_L56,0.973024,0.973009,9.6,6330,446,47.5
2,XB20_E2D8,0.973496,0.973490,332.0,31162,432,47.0
6,MLP20_L36,0.972981,0.972966,9.6,6330,446,47.0
9,MLP20_L26,0.972759,0.972742,9.6,6330,446,41.5
10,MLP20_L6,0.971693,0.971673,9.6,6330,446,41.0
4,DT20_D4,0.971328,0.971308,24.0,3694,432,35.0


## 40 %

The first cluster contains all six MLP models, two RF models (RF40_E6D8 and RF40_E4D8), and one DT model (DT40_D12)

In [17]:
cluster_1_40_bot = ['MLP40_L56','MLP40_L46','MLP40_L36', 'MLP40_L26', 'MLP40_L16', 'MLP40_L6','RF40_E6D8','RF40_E4D8','DT40_D12']

cluster_1_40_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_40_bot)]

In [18]:
bot_arduino_cluster_1_40 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_40_bot)]

df_bot_all_40 = pd.merge(cluster_1_40_bot_df, bot_arduino_cluster_1_40, on='Algo', how='inner')


In [19]:
df_bot_all_40.shape

(9, 6)

In [20]:

df_bot_all_40_tmp = df_bot_all_40.copy()
df_bot_all_40_tmp[['Latency', 'SRAM', 'FM']] = -df_bot_all_40[['Latency', 'SRAM', 'FM']]
df_bot_all_40_bc = borda_count(df_bot_all_40_tmp)
df_bot_all_40_bc[['Latency', 'SRAM', 'FM']] = -df_bot_all_40_bc[['Latency', 'SRAM', 'FM']]
df_bot_all_40_bc

,Algo,MCC,Kappa,Latency,SRAM,FM,Borda
2,DT40_D12,0.976575,0.976574,50.4,16480,652,53.5
0,RF40_E6D8,0.974817,0.974815,201.6,16556,652,41.0
3,MLP40_L46,0.974501,0.974501,9.6,6550,666,40.5
4,MLP40_L36,0.974478,0.974478,9.6,6550,666,40.0
5,MLP40_L56,0.974441,0.974441,9.6,6550,666,39.5
6,MLP40_L26,0.974421,0.974420,9.6,6550,666,39.0
7,MLP40_L16,0.974325,0.974325,9.6,6550,666,38.5
8,MLP40_L6,0.973163,0.973160,9.6,6550,666,33.0
1,RF40_E4D8,0.973182,0.973169,144.0,12096,652,17.0


# 60%

In [21]:
cluster_1_60_bot = ['DT60_D12']
cluster_1_60_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_60_bot)]
bot_arduino_cluster_1_60 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_60_bot)]
df_bot_all_60 = pd.merge(cluster_1_60_bot_df, bot_arduino_cluster_1_60, on='Algo', how='inner')


In [22]:
df_bot_all_60

,Algo,MCC,Kappa,Latency,SRAM,FM
0,DT60_D12,0.97942,0.979418,50.4,23044,872


## 100 %

. The first cluster contains all three RF models, all three ET models, both DT models, and four MLP models (MLP100_L46, MLP100_L56, MLP100_L16, and MLP100_L36)

In [40]:
cluster_1_100_bot = ['MLP100_L46','MLP100_L56','MLP100_L16','MLP100_L36','DT100_D12','RF100_E4D8','RF100_E6D8','RF100_E2D8','DT100_D4','ET100_E2D16','ET100_E6D8','ET100_E4D8']
cluster_1_100_bot_df = df_bot[df_bot['Algo'].isin(cluster_1_100_bot)]

In [41]:
bot_arduino_cluster_1_100 = bot_arduino[bot_arduino['Algo'].isin(cluster_1_100_bot)]

df_bot_all_100 = pd.merge(cluster_1_100_bot_df, bot_arduino_cluster_1_100, on='Algo', how='inner')


In [42]:
df_bot_all_100.shape

(12, 6)

In [43]:

df_bot_all_100_tmp = df_bot_all_100.copy()
df_bot_all_100_tmp[['Latency', 'SRAM', 'FM']] = -df_bot_all_100[['Latency', 'SRAM', 'FM']]
df_bot_all_100_bc = borda_count(df_bot_all_100_tmp)
df_bot_all_100_bc[['Latency', 'SRAM', 'FM']] = -df_bot_all_100_bc[['Latency', 'SRAM', 'FM']]
df_bot_all_100_bc

,Algo,MCC,Kappa,Latency,SRAM,FM,Borda
0,ET100_E2D16,0.978303,0.978295,8.0,5100,723,69.0
6,DT100_D12,0.986750,0.986732,33.6,23416,1172,68.0
7,DT100_D4,0.979489,0.979482,17.6,4444,1172,67.5
1,ET100_E6D8,0.969383,0.969287,8.0,5100,723,63.5
3,RF100_E4D8,0.982154,0.982145,159.2,16032,1172,59.0
2,ET100_E4D8,0.955665,0.955456,8.0,5100,723,58.0
5,RF100_E2D8,0.981386,0.981380,98.4,9760,1172,54.5
4,RF100_E6D8,0.982070,0.982062,220.0,21858,1172,51.5
8,MLP100_L46,0.970067,0.970025,9.6,7070,1186,41.0
9,MLP100_L56,0.956093,0.955627,9.6,7070,1186,35.5
